# Chatbot Modeling

## Import Libs

In [419]:
# General
import json
import string
import pickle
import numpy as np
import pandas as pd

# Preprocessing
from sklearn.preprocessing import LabelEncoder

# Deep Learning
import tensorflow as tf

# Word Preprocessing
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt') # Only needed for first time usage of NLTK

[nltk_data] Downloading package punkt to C:\Users\Judan Syamsul
[nltk_data]     Hadad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Load Files for Preprocessing and Modeling

### Intent

In [420]:
# Load json file
f = open('intent/intent.json', 'r')
intent_json = json.load(f)

In [421]:
# Create list from json
input = []
intent = []

for i in range(len(intent_json['intents'])):
    for user_input in intent_json['intents'][i]['input']:
        input.append(user_input)
        intent.append(intent_json['intents'][i]['intent'])

In [422]:
# Create dataframe from json
df = pd.DataFrame({
    'input': input,
    'intent' : intent
    # 'response' : response
})
df.head()

,input,intent
0,halo,sapa
1,hei,sapa
2,hey,sapa
3,hello,sapa
4,hallo,sapa


### Stopwords

In [423]:
# Define stemmer and stopword
factory = StemmerFactory()
stemmer = factory.create_stemmer()

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()

In [424]:
stopwords.remove('ok')
stopwords.remove('oh')
stopwords.remove('tidak')
stopwords.remove('ya')

### Slang

In [425]:
slang = pd.read_csv('lexicon/slang ke semi baku.csv')

slang_replace = {}
for i, row in enumerate(slang['slang']):
    slang_replace[row] = slang['formal'].iloc[i]

In [426]:
baku = pd.read_csv('lexicon/slang ke baku.csv')

std_word_replace = {}
for i, row in enumerate(baku['slang']):
    std_word_replace[row] = baku['baku'].iloc[i]

## Preprocessing

In [427]:
# Create text cleaning function
def clean_text(text):
    new_text = []
    text = text.lower() # Lowercase
    # Loop each word in a sentence
    for kata in text.split(): 
        # Keep word not in slang or standard word
        if kata not in (slang_replace|std_word_replace): 
            new_text.append(kata) 
        # Replace non-formal word with standard word
        elif kata in std_word_replace:
            new_text+=std_word_replace[kata].split() 
        # Replace slang with standard word
        elif kata in slang_replace:
            for kata_slang in slang_replace[kata].split():
                new_text.append(std_word_replace.get(kata_slang, kata_slang))
    # Join words without stopwords after stemming
    new_text = ' '.join(
        stemmer.stem(word) for word in new_text if word not in stopwords
    )
    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))
    return new_text

In [428]:
kalimat = 'Aku laper banget gatau maunya makan jeruk apa lagi meniru-niru... daah ngelamar'
clean_text(kalimat)

'aku lapar banget tidak tahu mau makan jeruk apa tiru deh lamar'

In [429]:
df['clean_input'] = df['input'].apply(clean_text)
df['clean_input'].head()

0     halo
1      hei
2      hai
3    hello
4     halo
Name: clean_input, dtype: object

## Create Corpus

In [430]:
# Create corpus
words = set([
    word for word in df['clean_input'] for word in word_tokenize(word)
    ]
)

In [431]:
corpus_size = len(words)

In [432]:
# Calculate word length of each row
df['length'] = df['clean_input'].apply(word_tokenize).apply(len)

In [433]:
sequence_length = int(round(df['length'].max(),0))

In [434]:
corpus_size, sequence_length

(314, 8)

## Label Encoding

In [435]:
# Label encoding
le = LabelEncoder()
y_train = le.fit_transform(df['intent'])
y_train = tf.keras.utils.to_categorical(y_train)

In [436]:
le.classes_

array(['CV', 'biaya', 'daftar', 'dokumen', 'error upload', 'ktp',
       'link sosmed', 'lowongan', 'lupa password', 'nama', 'pas foto',
       'pengantar', 'penutup', 'qualification', 'responsibilities',
       'salary', 'sapa', 'sertifikat', 'skck', 'timeline', 'training',
       'transkrip-ijazah'], dtype=object)

In [437]:
len(le.classes_)

22

## Modeling

### Text Vectorization

In [438]:
textvect = tf.keras.layers.TextVectorization(
    max_tokens=corpus_size,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='int',
    output_sequence_length=sequence_length
)
textvect.adapt(df['clean_input'])

In [439]:
tes = 'saya mau daftar rekrutmen'
textvect(clean_text(tes))

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([18, 12,  9,  0,  0,  0,  0,  0], dtype=int64)>

### Word Embedding

In [440]:
embedding = tf.keras.layers.Embedding(
    input_dim=corpus_size,
    output_dim=16,
    input_length=sequence_length,
    embeddings_initializer='uniform'
)

In [441]:
embedding(textvect(clean_text(tes)))

<tf.Tensor: shape=(8, 16), dtype=float32, numpy=
array([[-0.04628922,  0.04706401,  0.04509645,  0.02850921,  0.01524427,
         0.00050949, -0.00790543, -0.02300199, -0.03255873,  0.04800328,
        -0.0329253 , -0.01142097, -0.04734316,  0.03545928, -0.00174334,
        -0.03832507],
       [ 0.04382794, -0.04006261, -0.01430545, -0.02373762, -0.02264904,
        -0.03883886,  0.00053902,  0.03389079,  0.02255959,  0.03155177,
        -0.03520573,  0.0429204 ,  0.00635047, -0.02780977, -0.03219408,
        -0.01572938],
       [-0.01248238, -0.03902546,  0.03356062,  0.03509945,  0.00406519,
         0.03886166,  0.04373487,  0.00882006, -0.00550128, -0.02226261,
         0.00692222,  0.03379739, -0.01411591, -0.0166749 ,  0.02571679,
        -0.04267466],
       [-0.0435619 ,  0.0347769 , -0.03324093, -0.04556705, -0.0256171 ,
        -0.02406508,  0.00504791, -0.04256728,  0.03735918,  0.00948579,
        -0.02162124,  0.0078191 ,  0.0457275 ,  0.02229733, -0.0395906 ,
        -

### Layering

In [442]:
# Create model
input = tf.keras.layers.Input(shape=(1,), dtype='string')
hidden_layer = textvect(input)
hidden_layer = embedding(hidden_layer)
hidden_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16))(hidden_layer)
output = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(hidden_layer)
model = tf.keras.Model(inputs=input, outputs=output)

# Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy']
)

### Training

In [443]:
hist = model.fit(df['clean_input'], y_train, epochs=100, verbose=0)
pd.DataFrame(hist.history).tail()

,loss,categorical_accuracy
95,0.049314,0.995772
96,0.047498,0.995772
97,0.046245,0.995772
98,0.045115,0.995772
99,0.044239,0.995772


### Evaluating Model

In [444]:
model.evaluate(df['clean_input'], y_train)

15/15 [==============================] - 1s 4ms/step - loss: 0.0433 - categorical_accuracy: 0.9958


[0.04331756383180618, 0.9957716464996338]

In [445]:
y_train_df = pd.DataFrame(y_train, columns=le.classes_)
y_train_df['intent'] = y_train_df.idxmax(axis=1)

model_pred = model.predict(df['clean_input'])
model_pred = pd.DataFrame(model_pred, columns=le.classes_)
model_pred['intent'] = model_pred.idxmax(axis=1)

from sklearn.metrics import classification_report
print(classification_report(y_train_df['intent'], model_pred['intent']))

                  precision    recall  f1-score   support

              CV       1.00      1.00      1.00        12
           biaya       1.00      1.00      1.00        27
          daftar       1.00      0.95      0.98        22
         dokumen       0.96      1.00      0.98        23
    error upload       1.00      1.00      1.00         9
             ktp       1.00      1.00      1.00        16
     link sosmed       1.00      1.00      1.00        17
        lowongan       1.00      1.00      1.00        17
   lupa password       1.00      1.00      1.00        21
            nama       1.00      1.00      1.00         9
        pas foto       1.00      1.00      1.00        16
       pengantar       1.00      1.00      1.00        10
         penutup       1.00      1.00      1.00        48
   qualification       1.00      0.94      0.97        18
responsibilities       1.00      1.00      1.00        21
          salary       1.00      1.00      1.00        37
            s

### Function Covering End-to-End Process

In [446]:
def bot_response(text):
    """Take text as function input then predict using model. Return response based on highest probability using numpy argmax    
    """
    text = clean_text(text)
    pred = model.predict([text])
    res = le.classes_[pred.argmax()] # Get the index with highest probability
    i = 0
    try:
        if textvect(text).numpy().max() > 1: # If the input is known word(s)
            while i < len(intent_json['intents']):
                if res == intent_json['intents'][i]['intent']:
                    response = intent_json['intents'][i]['response']
                    break
                else:
                    i+=1
        else: # If only unknown word(s)
            response = ['Maaf, Kak. Aku tidak mengerti...']
    except: # If empty string or any error occured
        response = ['Maaf, Kak. Aku tidak mengerti...']

    # For debugging only
    dict_temp = []
    for i in range(len(pred[0])):
        temp = {le.classes_[i]: pred[0][i]}
        dict_temp.append(temp)
    print(dict_temp)
    print(le.classes_[pred.argmax()])

    return print(np.random.choice(response))

In [447]:
tes1 = 'aku mau daftar rekrutmen'
clean_text(tes1)

'aku mau daftar rekrutmen'

In [448]:
model.predict([clean_text(tes1)])

array([[1.0277510e-03, 2.6737080e-06, 9.5001376e-01, 1.2308438e-04,
        2.2947192e-02, 3.4236927e-03, 4.5484561e-03, 1.4445170e-06,
        1.0642342e-05, 1.3235601e-04, 3.3241706e-03, 4.7060268e-05,
        4.9949416e-05, 1.2549878e-02, 1.7944700e-05, 1.4382680e-08,
        1.0489405e-05, 9.2753144e-05, 1.6361320e-05, 1.8386311e-04,
        2.4765634e-04, 1.2289513e-03]], dtype=float32)

In [449]:
model.predict([clean_text(tes1)]).argmax()

2

In [450]:
le.classes_[model.predict([clean_text(tes1)]).argmax()]

'daftar'

In [451]:
le.classes_[model.predict([clean_text(tes1)]).argmax()] == intent_json['intents'][2]['intent']

True

In [452]:
intent_json['intents'][2]['response']

['Untuk mengikuti rekrutmen ini, mohon ikuti langkah berikut ya, Kak:\n1. Baca tata cara melamar di https://rekrutmen.fiktif.id/tatacara\n2. Buat akun di https://rekrutmen.fiktif.id/registrasi menggunakan alamat email dan nomor telepon aktif milik pribadi\n3. Lengkapi formulir pendaftaran dan upload seluruh dokumen yang dibutuhkan dan tunggu pengumuman tahap berikutnya di https://rekrutmen.fiktif.id/news atau email masing-masing']

In [453]:
bot_response('aku mau daftar rekrutmen')

[{'CV': 0.001027751}, {'biaya': 2.673708e-06}, {'daftar': 0.95001376}, {'dokumen': 0.00012308438}, {'error upload': 0.022947192}, {'ktp': 0.0034236927}, {'link sosmed': 0.004548456}, {'lowongan': 1.444517e-06}, {'lupa password': 1.0642342e-05}, {'nama': 0.00013235601}, {'pas foto': 0.0033241706}, {'pengantar': 4.7060268e-05}, {'penutup': 4.9949416e-05}, {'qualification': 0.012549878}, {'responsibilities': 1.79447e-05}, {'salary': 1.438268e-08}, {'sapa': 1.0489405e-05}, {'sertifikat': 9.2753144e-05}, {'skck': 1.636132e-05}, {'timeline': 0.00018386311}, {'training': 0.00024765634}, {'transkrip-ijazah': 0.0012289513}]
daftar
Untuk mengikuti rekrutmen ini, mohon ikuti langkah berikut ya, Kak:
1. Baca tata cara melamar di https://rekrutmen.fiktif.id/tatacara
2. Buat akun di https://rekrutmen.fiktif.id/registrasi menggunakan alamat email dan nomor telepon aktif milik pribadi
3. Lengkapi formulir pendaftaran dan upload seluruh dokumen yang dibutuhkan dan tunggu pengumuman tahap berikutnya di 

### Saving Model

In [454]:
# import pickle
# pickle.dump(le, open('saved_model/encoder.pkl', 'wb'))

In [455]:
# pickle.dump({'config': textvect.get_config(),
#              'weights': textvect.get_weights()}
#             , open("saved_model/textvect.pkl", "wb"))

In [456]:
# model.save('saved_model/model', save_traces=True)

## Inference

In [457]:
textvect('ok')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([233,   0,   0,   0,   0,   0,   0,   0], dtype=int64)>

In [458]:
clean_text('ok')

'ok'

In [459]:
textvect('ok').numpy().max() > 1

True

In [460]:
bot_response('ok')

[{'CV': 4.8556896e-05}, {'biaya': 0.0016888581}, {'daftar': 0.0002604772}, {'dokumen': 0.0002836671}, {'error upload': 3.1121624e-06}, {'ktp': 3.0344603e-05}, {'link sosmed': 0.00011439555}, {'lowongan': 7.0523245e-05}, {'lupa password': 0.0001878958}, {'nama': 0.00010687495}, {'pas foto': 0.00022719592}, {'pengantar': 4.06198e-06}, {'penutup': 0.979364}, {'qualification': 6.989686e-05}, {'responsibilities': 3.0208204e-05}, {'salary': 0.003446692}, {'sapa': 0.0004232541}, {'sertifikat': 1.7639411e-06}, {'skck': 9.509653e-07}, {'timeline': 0.0049114237}, {'training': 0.008648678}, {'transkrip-ijazah': 7.711706e-05}]
penutup
Terima kasih telah bertanya. Senang bisa membantumu :D


In [461]:
bot_response('ya')

[{'CV': 4.7141562e-05}, {'biaya': 0.0009850519}, {'daftar': 0.0005631299}, {'dokumen': 0.00027140466}, {'error upload': 4.8231677e-06}, {'ktp': 0.00012890693}, {'link sosmed': 0.00014950214}, {'lowongan': 6.2801104e-05}, {'lupa password': 0.0004953896}, {'nama': 4.6454494e-05}, {'pas foto': 0.00035225888}, {'pengantar': 4.7120366e-06}, {'penutup': 0.9600971}, {'qualification': 3.105466e-05}, {'responsibilities': 1.2725958e-05}, {'salary': 0.0030748355}, {'sapa': 0.0017374876}, {'sertifikat': 7.2974456e-07}, {'skck': 3.4849077e-06}, {'timeline': 0.010866457}, {'training': 0.020781834}, {'transkrip-ijazah': 0.0002827644}]
penutup
Terima kasih telah bertanya. Senang bisa membantumu :D


In [462]:
testext = 'jeruk'
clean_text(testext), textvect(clean_text(testext))

('jeruk',
 <tf.Tensor: shape=(8,), dtype=int64, numpy=array([1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)>)

In [463]:
textvect(clean_text('laper pengen makan jeruk')).numpy().max() > 1

False

In [464]:
bot_response('oh ok')

[{'CV': 5.494427e-05}, {'biaya': 0.0019707626}, {'daftar': 0.0001226821}, {'dokumen': 0.00012766401}, {'error upload': 2.017975e-06}, {'ktp': 3.7075552e-06}, {'link sosmed': 8.129155e-05}, {'lowongan': 6.468408e-05}, {'lupa password': 5.9820708e-05}, {'nama': 0.00025218242}, {'pas foto': 0.00015741281}, {'pengantar': 3.142627e-06}, {'penutup': 0.9885029}, {'qualification': 9.794857e-05}, {'responsibilities': 5.0033676e-05}, {'salary': 0.0030904093}, {'sapa': 0.00010629905}, {'sertifikat': 4.5596844e-06}, {'skck': 1.8435064e-07}, {'timeline': 0.0012137752}, {'training': 0.0040134178}, {'transkrip-ijazah': 2.0086578e-05}]
penutup
Terima kasih telah bertanya. Senang bisa membantumu :D


In [465]:
bot_response('ada lowongan apa nih di pt fiktif')

[{'CV': 3.9208953e-05}, {'biaya': 0.00058952114}, {'daftar': 7.2854473e-06}, {'dokumen': 0.0006187087}, {'error upload': 4.6430843e-05}, {'ktp': 0.00076969404}, {'link sosmed': 0.017951695}, {'lowongan': 0.93188035}, {'lupa password': 0.00050851796}, {'nama': 0.016155807}, {'pas foto': 8.457471e-07}, {'pengantar': 0.018493598}, {'penutup': 1.5611648e-05}, {'qualification': 0.00013207732}, {'responsibilities': 0.0038784712}, {'salary': 0.00035872168}, {'sapa': 0.0032339848}, {'sertifikat': 1.566238e-05}, {'skck': 0.0018240911}, {'timeline': 0.00030900526}, {'training': 3.1765825e-07}, {'transkrip-ijazah': 0.0031703361}]
lowongan
Saat ini PT Fiktif sedang membuka lowongan untuk posisi Data Analyst, Data Scientist, dan Data Engineer. Yuk daftar lewat https://rekrutmen.fiktif.id/


In [466]:
bot_response('ingin')

[{'CV': 6.2850646e-05}, {'biaya': 0.014318895}, {'daftar': 0.0005963996}, {'dokumen': 0.0013402742}, {'error upload': 9.800359e-06}, {'ktp': 0.0011694937}, {'link sosmed': 0.0003467348}, {'lowongan': 0.00026314723}, {'lupa password': 0.0011491058}, {'nama': 7.6258744e-05}, {'pas foto': 0.0003529772}, {'pengantar': 1.4296612e-05}, {'penutup': 0.77157295}, {'qualification': 9.684284e-05}, {'responsibilities': 6.669808e-05}, {'salary': 0.008857448}, {'sapa': 0.007472787}, {'sertifikat': 7.8815754e-07}, {'skck': 2.7136515e-05}, {'timeline': 0.13452181}, {'training': 0.057019144}, {'transkrip-ijazah': 0.00066418975}]
penutup
Maaf, Kak. Aku tidak mengerti...


In [467]:
bot_response('pas foto hilang gimana ya')

[{'CV': 0.002115128}, {'biaya': 4.1439136e-05}, {'daftar': 0.013877773}, {'dokumen': 3.5477682e-05}, {'error upload': 0.00030848465}, {'ktp': 0.00025191595}, {'link sosmed': 0.00029500926}, {'lowongan': 4.035449e-06}, {'lupa password': 0.0010332903}, {'nama': 2.8982919e-05}, {'pas foto': 0.97517633}, {'pengantar': 4.2841006e-05}, {'penutup': 0.00032565076}, {'qualification': 0.0028871223}, {'responsibilities': 0.00016922146}, {'salary': 2.033037e-06}, {'sapa': 4.0677416e-05}, {'sertifikat': 0.002502428}, {'skck': 0.00040781902}, {'timeline': 2.981867e-07}, {'training': 0.00041789468}, {'transkrip-ijazah': 3.61401e-05}]
pas foto
Pas foto merupakan dokumen wajib pendaftaran ya, Kak.

Berikut ketentuan pas foto yang harus dipenuhi:
- Background berwarna biru dengan pakaian formal (kemeja dan/atau jas)
- Foto berukuran 3x4 dan berwarna
- Disarankan untuk menggunakan foto terbaru
- Upload foto di https://rekrutmen.fiktif.id/dokumen dengan ukuran file tidak lebih dari 1 MB dengan format file

ktp, pas foto, password masih ketuker2

In [468]:
le.classes_

array(['CV', 'biaya', 'daftar', 'dokumen', 'error upload', 'ktp',
       'link sosmed', 'lowongan', 'lupa password', 'nama', 'pas foto',
       'pengantar', 'penutup', 'qualification', 'responsibilities',
       'salary', 'sapa', 'sertifikat', 'skck', 'timeline', 'training',
       'transkrip-ijazah'], dtype=object)

In [469]:
textvect('loker')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([153,   0,   0,   0,   0,   0,   0,   0], dtype=int64)>

In [470]:
textvect('wis')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)>

In [471]:
data = {'user_input':'hai hai'}
data = json.dumps(data)
loaded_data = json.loads(data)

In [472]:
type(loaded_data)

dict

In [473]:
tespred = str(model.predict([clean_text(loaded_data['user_input'])]).argmax())

In [474]:
tes_res = {'pred':tespred}
tes_res = json.dumps(tes_res)
loaded_res = json.loads(tes_res)

In [475]:
le.classes_[int(loaded_res['pred'])]

'transkrip-ijazah'

In [476]:
loaded_data['user_input']

'hai hai'

In [477]:
1000

1000

In [478]:
str(1000)

'1000'

In [479]:
int(str(1000))

1000

In [480]:
le.classes_[int(str(10))]

'pas foto'

In [481]:
textvect('loker')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([153,   0,   0,   0,   0,   0,   0,   0], dtype=int64)>

In [499]:
user_input = 'apa kualifikasi engineer'
clean_input = clean_text(user_input)
pred = model.predict([clean_input]).argmax()
if textvect(clean_input).numpy().max() <= 1:
    label_idx = 1000
else:
    label_idx = pred 
if label_idx != 1000:
    i = 0
    while i < len(intent_json['intents']):
        if le.classes_[label_idx] == intent_json['intents'][i]['intent'] and (le.classes_[label_idx] != 'responsibilities' or le.classes_[label_idx] != 'qualification'):
            response = intent_json['intents'][i]['response']
            break
        elif le.classes_[label_idx] == 'responsibilities':
            if any(x in clean_input for x in ['scientist', 'ds']):
                response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Scientist:\n- Merancang dan mengembangkan berbagai solusi Machine Learning dan Deep Learning untuk meningkatkan pengalaman pengguna bagi konsumen\n- Berkolaborasi dengan seluruh elemen bisnis dan bertanggung jawab untuk merencanakan solusi end-to-end berbasis data untuk menyelesaikan permasalahan bisnis\n- Menjadi thinking partner bagi stakeholder lain untuk memperbaiki alur perjalanan data dan penggunaannya dalam operasional perusahaan, misal merancang proses feedback-loop atau human-in-the-loop untuk meningkatkan performa model secara berkelanjutan"]
                break
            elif any(x in clean_input for x in ['engineer', 'de']):
                response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Engineer:\n- Membangun dan menjaga end-to-end data pipeline dari input dan output heterogen\n- Menangani dan mengelola data warehouse\n- Membantu tim mentransformasikan data (ETL) dan mengembangkan proses ETL dari beberapa sumber\n- Menganalisis dan mengorganisir data mentah \n- Memastikan kualitas data dan integrasi data"]
                break
            elif any(x in clean_input for x in ['analyst', 'da', 'analis']):
                response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Analyst:\n- Mengumpulkan dan menyediakan data untuk membantu stakeholder lain meningkatkan metrik bisnis perusahaan dan retensi pelanggan\n- Menganalisis data untuk menemukan insight yang dapat ditindaklanjuti seperti membuat funnel conversion analysis, cohort analysis, long-term trends, user segmentation, dan dapat membantu meningkatkan kinerja perusahaan dan mendukung pengambilan keputusan yang lebih baik\n - Mengidentifikasi kebutuhan dan peluang bisnis berdasarkan data yang tersedia"]
                break
        elif le.classes_[label_idx] == 'qualification':
            if any(x in clean_input for x in ['scientist', 'ds']):
                response = ["Untuk posisi Data Scientist ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pemahaman mendasar tentang Statistika Analitik, Machine Learning, Deep Learning untuk menyelesaikan permasalahan bisnis\n3. Memiliki pengalaman kerja di bidang Data Science selama 1-3 tahun\n4. Memiliki pemahaman dan pengalaman tentang Big Data\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                break
            elif any(x in clean_input for x in ['engineer', 'de']):
                response = ["Untuk posisi Data Engineer ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pengalaman bekerja dengan tools untuk ETL seperti AWS Glue, SSIS, Informatica, dll.\n3. Memiliki pengalaman kerja di bidang Data Engineer selama 1-3 tahun\n4. Memiliki pemahaman yang baik tentang ETL, SQL, dan noSQL\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                break
            elif any(x in clean_input for x in ['analyst', 'da', 'analis']):
                response = ["Untuk posisi Data Analyst ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pemahaman mendasar tentang Statistika Analitik dan Inferensial untuk mencari peluang bisnis\n3. Memiliki pengalaman kerja di bidang Data Analyst selama 1-3 tahun\n4. Memiliki pemahaman dan pengalaman tentang Big Data serta visualisasi dengan tools seperti Power BI, Tableau, dll.\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                break
        else:
            i+=1
else:
    response = ['Maaf, Kak. Aku tidak mengerti chatnya...']

print(np.random.choice(response))

Untuk posisi Data Engineer ada beberapa kualifikasi yang harus dipenuhi:
1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan
2. Memiliki pengalaman bekerja dengan tools untuk ETL seperti AWS Glue, SSIS, Informatica, dll.
3. Memiliki pengalaman kerja di bidang Data Engineer selama 1-3 tahun
4. Memiliki pemahaman yang baik tentang ETL, SQL, dan noSQL
5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik


In [483]:
le.classes_[label_idx]

'sertifikat'